In [1]:
import os
import numpy as np
import cv2
from pyswarm import pso

In [2]:
import os
import numpy as np
import cv2
import pandas as pd
from pyswarm import pso
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# --- Paths ---
csv_path = "D:/B.Tech/8th Sem/CI Lab/Project/Cylone_Intensity_IndianOcean/label.csv"
image_dir = "D:/B.Tech/8th Sem/CI Lab/Project/Cylone_Intensity_IndianOcean/Infrared"

In [3]:
# --- 1. Load CSV ---
df = pd.read_csv(csv_path)
df['filepath'] = df['Filename'].apply(lambda x: os.path.join(image_dir, x))
print(f"Total samples from CSV: {len(df)}")

Total samples from CSV: 15101


In [4]:
# --- 2. Extract raw features from images ---
def extract_image_features(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (64, 64)).astype(np.float32) / 255.0
    return image.flatten()

X = np.array([extract_image_features(p) for p in df['filepath']])
y = df['Vmax'].values

print(f"Raw feature shape: {X.shape}")  # (samples, features)
print(f"Labels shape: {y.shape}")

Raw feature shape: (15101, 4096)
Labels shape: (15101,)


In [5]:
# --- 4. Load saved selected feature mask ---
selected_mask = np.load("selected_features_mask.npy")
X_selected = X[:, selected_mask]
print(f"Selected features count: {np.sum(selected_mask)}")
print(f"Feature shape after PSO selection: {X_selected.shape}")

# --- 5. Train-test split ---
# split: train and test
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.2, random_state=42)


# Print shapes to confirm
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")

Selected features count: 2141
Feature shape after PSO selection: (15101, 2141)
X_test: (3021, 2141), y_test: (3021,)
X_train: (12080, 2141), y_train: (12080,)


In [6]:
# --- Convert to Tensors ---
def to_loader(X, y):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1)
    return DataLoader(TensorDataset(X_tensor, y_tensor), batch_size=32, shuffle=True)

train_loader = to_loader(X_train, y_train)
test_loader = to_loader(X_test, y_test)


## SVR

In [7]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# --- Scale features ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Train SVR model ---
svr_model = SVR(kernel='rbf', C=100, gamma='scale', epsilon=0.1)
svr_model.fit(X_train_scaled, y_train)

# --- Predict ---
y_pred_train = svr_model.predict(X_train_scaled)

# --- Train Evaluation ---
mae = mean_absolute_error(y_train, y_pred_train)
mse = mean_squared_error(y_train, y_pred_train)
r2 = r2_score(y_train, y_pred_train)

print(f"Train Evaluation Results:")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R2 Score: {r2:.4f}")

# --- Test Evaluation ---
y_pred = svr_model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Test Evaluation Results:")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R2 Score: {r2:.4f}")


Train Evaluation Results:
Mean Absolute Error (MAE): 3.65
Mean Squared Error (MSE): 110.99
R2 Score: 0.9067
Test Evaluation Results:
Mean Absolute Error (MAE): 14.92
Mean Squared Error (MSE): 416.56
R2 Score: 0.6448


## Grid Serach

In [ ]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV

# --- Scale features ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ------------------------------------------------------------------
# 1. Set up the parameter grid
#    – Feel free to widen / tighten these ranges.
# ------------------------------------------------------------------
param_grid = {
    'C':      [1, 10, 100, 300],
    'gamma':  ['scale', 'auto', 0.01, 0.001],
    'epsilon':[0.01, 0.05, 0.1, 0.2]
}

# ------------------------------------------------------------------
# 2. Configure the SVR and GridSearchCV
# ------------------------------------------------------------------
base_svr   = SVR(kernel='rbf')
grid_search = GridSearchCV(
    estimator = base_svr,
    param_grid = param_grid,
    cv = 5,                        # 5‑fold cross‑validation
    scoring = 'neg_mean_squared_error',
    n_jobs = -1,                   # use all CPU cores
    verbose = 2                    # 0 = silent, 1 = minimal, 2 = detailed
)

# ------------------------------------------------------------------
# 3. Run the grid search
# ------------------------------------------------------------------
grid_search.fit(X_train_scaled, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best CV MSE:", -grid_search.best_score_)

# ------------------------------------------------------------------
# 4. Evaluate the best model on train & test sets
# ------------------------------------------------------------------
best_svr = grid_search.best_estimator_

y_pred_train = best_svr.predict(X_train_scaled)
y_pred_test  = best_svr.predict(X_test_scaled)

def report(y_true, y_pred, split):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2  = r2_score(y_true, y_pred)
    print(f"\n{split} Results  |  MAE: {mae:.3f}  MSE: {mse:.3f}  R²: {r2:.4f}")

report(y_train, y_pred_train, "Train")
report(y_test,  y_pred_test,  "Test")

Fitting 5 folds for each of 64 candidates, totalling 320 fits
